In [5]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.metrics import BinaryAccuracy,AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler
from utils import build_lrfn

In [7]:
BATCH_SIZE = 4 
EPOCHS = 5
TRAINING_DATA_SIZE = 33126
TEST_DATA_SIZE = 10982
IMAGE_SIZE = [512,512]
RESIZE_SHAPE = [512,512]
VALIDATION_SPLIT = 0.2
N_SPLITS = 5
LEARNING_RATE = 1e-5
LR_MAX = 0.0004 
LR_MIN = 1e-6 
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 1
LR_EXP_DECAY = 0.8
scheduler = build_lrfn(lr_start=LEARNING_RATE,
                       lr_max=LR_MAX,
                       lr_min=LR_MIN,
                       lr_rampup_epochs=LR_RAMPUP_EPOCHS,
                       lr_sustain_epochs=0,
                       lr_exp_decay=LR_EXP_DECAY)
test = tf.io.gfile.glob('./Dataset/tfrecords/test*.tfrec')

In [ ]:
oof_val_predictions = np.zeros((TRAINING_DATA_SIZE,))
test_preds = np.zeros((TEST_DATA_SIZE,N_SPLITS))
test_idnums = np.zeros((TEST_DATA_SIZE,))



for i in range(N_SPLITS):

    training_file = tf.io.gfile.glob('train*.tfrec')[i]
    validation_file = tf.io.gfile.glob('validation*.tfrec')[i]

    save_path = f'./model/saved_models/EfficientNetB5_fold_{i+1}.h5'

    print(f"Save model in path: {save_path}")

    callbacks = [ 
        ModelCheckpoint(save_path,monitor='val_auc',verbose=1,save_best_only=True),
        LearningRateScheduler(scheduler,verbose=1)
    ]

    Data = Dataset(
        train_files = training_file,
        test_files = test,
        validation_files = validation_file,
        validation_split = VALIDATION_SPLIT,
        image_size = IMAGE_SIZE,
        shuffle = SHUFFLE,
        dataset_size = TRAINING_DATA_SIZE,
        batch_size = BATCH_SIZE,
        resize_shape = RESIZE_SHAPE
    )

    Model = ModelCreation(
        architecture = 'efficientnet',
        learning_rate = LEARNING_RATE,
        input_shape = (*RESIZE_SHAPE,3),
        output_shape = 1, 
        optimizer = Adam,
        metric = [BinaryAccuracy(),AUC()],
        loss = BinaryCrossentropy,
        linear = True,
        verbose = True
    )

    training = Data.get_training_dataset()
    validation = Data.get_validation_dataset()


    history = Model.model.fit(training,
                            steps_per_epoch = Data.get_train_steps_per_epoch(),
                            epochs = EPOCHS,
                            validation_data = validation,
                            validation_steps = Data.get_validation_steps_per_epoch(),
                            verbose = 1,
                            callbacks = Model.inject_callbacks(callbacks))

    print(f"Loading model from path: {save_path}")

    Load_model = load_model(save_path)

    validation_images = validation_data.map(lambda image,label: image)
    
    probabilities = Load_model.predict(validation_images)

    oof_val_predictions[val_idx] = np.concatenate(probabilities)

    #Get test set predictions in fold 

    test_data = Data.get_test_dataset()

    test_images = test_data.map(lambda image,idnum : image)
    test_probabilities = Load_model.predict(test_images)

    test_preds[:,i] = np.concatenate([test_probabilities])

    test_ids = test_data.map(lambda image,idnum: idnum).unbatch()
    test_ids_images = next(iter(test_ids.batch(TEST_DATA_SIZE))).numpy().astype('U')

    test_idnums[:] = test_ids_images

